In [19]:
import numpy as np
import re
import h5py as h5
import matplotlib.pyplot as plt

#HDF file generate from Jon's scripts comprising matched candidate labels and filenames.
input_file = '/projects/nikhil/miccai/input_data_comb/data_t300_adcn.h5'

# Run this script for each split, side combination:
split = 'test'
side = 'l'

input_data = h5.File(input_file, 'r')
input_HC_all = input_data['{}_{}_data'.format(side, split)][:]
input_classes_all = input_data['{}_{}_classes'.format(side, split)][:]
input_files_all = input_data['{}_{}_files'.format(side, split)][:]
input_data.close()

id_participant = re.compile(r"""
 (?<=ADNI_)      # Match the first string after ADNI_
 (.*?)          # Lazy quantifier so it only grabs the first immediate match.
 (?=_MR)        # End at the _MR
""", re.VERBOSE)

In [20]:
# If creating pairwise datasets (for classification tasks)
# Pick the Dx group to ignore AD:0, CN:1, MCI:2 (-1 basically picks all 3 groups)
Dx_excl = -1
input_classes = input_classes_all[input_classes_all!=Dx_excl]
input_files = input_files_all[input_classes_all!=Dx_excl]
input_HC = input_HC_all[input_classes_all!=Dx_excl]

print len(input_files)
print input_HC.shape

8212
(8212, 11427)


In [21]:
# Generate dictionary {subject_id:[candidate_labels]}
import collections
import scipy.stats as stats

subject_idx=[]
unique_subject_idx = []
subject_vol_dict = collections.OrderedDict()
subject_class_dict = collections.OrderedDict()

#find volume indices for each unique subject
seen = set([])
for j, f in enumerate(input_files):
    subject_id = re.search(id_participant, f).group(0)
    subject_idx.append(subject_id)
    if subject_id not in subject_vol_dict:
        subject_vol_dict[subject_id]=[]
        subject_class_dict[subject_id]=[]
    
    subject_vol_dict[subject_id].append(input_HC[j])
    subject_class_dict[subject_id].append(input_classes[j])

    if subject_id not in seen:
        unique_subject_idx.append(subject_id)
        seen.add(subject_id)
    
print "Dictionary length: " + str(len(subject_vol_dict))
    
print len(unique_subject_idx)

Dictionary length: 110
110


In [22]:
#Save Dictionaries

sub_HC_vol_left_file = '/projects/nikhil/ADNI_prediction/input_datasets/HC/subject_HC_vol_dictionary_heldout_left.pkl'
sub_HC_vol_right_file = '/projects/nikhil/ADNI_prediction/input_datasets/HC/subject_HC_vol_dictionary_heldout_right.pkl'

import pickle
f = open(sub_HC_vol_left_file, 'wb')
pickle.dump(subject_vol_dict, f)
f.close()
